## 데이터 전처리

### 수치형
사상자수, 사망자수, 중상자수, 경상자수, 부상신고자수

### 범주형
주야, 요일, 사망자수, 사상자수, 중상자수, 경상자수, 부상신고자수, 발생지시도, 발생지시군구, 
사고유형_대분류, 사고유형_중분류, 법규위반, 도로형태_대분류,
도로형태, 당사자종별_1당_대분류, 당사자종별_2당_대분류

In [290]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [358]:
category = ['주야', '요일', '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', 
            '도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류']

x_train_num = pd.read_csv('./교통사망사고정보/Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949', 
                              usecols=['사상자수', '사망자수', '중상자수', '경상자수','부상신고자수'])

x_train_cat = pd.read_csv('./교통사망사고정보/Train_교통사망사고정보(12.1~17.6).csv',encoding='cp949',
                               usecols=['주야', '요일', '사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수', 
                                        '발생지시도', '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', 
                                        '도로형태_대분류', '도로형태', '당사자종별_1당_대분류', '당사자종별_2당_대분류'])


In [359]:
# 당사자종별_2당_대분류 0, 빈칸 -> 없음 처리
x_train_cat['당사자종별_2당_대분류'].replace(['0', ''], '없음')


0        보행자
1        승용차
2        보행자
3        이륜차
4        화물차
5        승용차
6        화물차
7        보행자
8        이륜차
9        승용차
10        없음
11       보행자
12        없음
13        없음
14       보행자
15        없음
16       이륜차
17       보행자
18       보행자
19       보행자
20        없음
21       보행자
22       화물차
23       보행자
24        없음
25       보행자
26        없음
27       이륜차
28        없음
29       승용차
        ... 
25007     없음
25008     없음
25009    보행자
25010    보행자
25011    보행자
25012    보행자
25013    화물차
25014     없음
25015     없음
25016    보행자
25017    보행자
25018    승용차
25019    승용차
25020    승용차
25021    보행자
25022    보행자
25023     없음
25024     없음
25025    승용차
25026     없음
25027    특수차
25028    보행자
25029     없음
25030     없음
25031    보행자
25032     없음
25033    승용차
25034    보행자
25035    승용차
25036    보행자
Name: 당사자종별_2당_대분류, Length: 25037, dtype: object


In [360]:
encoder = LabelEncoder()

def transform_label(value, col):    
    print(value)
    try:
        encoder.fit(x_train_cat[col].astype(str).unique())
        return encoder.transform(value)
    except Exception as ex:
        print('ex',ex,'value', value)
    
    
for col in category:
    print(col)
    x_train_cat[col] = transform_label(x_train_cat[col].astype(str).values, col)

주야
['야간' '야간' '야간' ... '야간' '야간' '야간']
요일
['금' '금' '금' ... '일' '일' '일']
발생지시도
['경기' '전남' '충남' ... '서울' '전북' '서울']
발생지시군구
['성남시' '곡성군' '서산시' ... '서초구' '정읍시' '은평구']
사고유형_대분류
['차대사람' '차대차' '차대사람' ... '차대사람' '차대차' '차대사람']
사고유형_중분류
['횡단중' '추돌' '차도통행중' ... '횡단중' '정면충돌' '차도통행중']
법규위반
['안전운전 의무 불이행' '안전운전 의무 불이행' '안전운전 의무 불이행' ... '안전운전 의무 불이행' '중앙선 침범'
 '안전운전 의무 불이행']
도로형태_대분류
['단일로' '단일로' '단일로' ... '교차로' '단일로' '단일로']
도로형태
['기타단일로' '기타단일로' '기타단일로' ... '교차로부근' '기타단일로' '기타단일로']
당사자종별_1당_대분류
['승용차' '승용차' '승용차' ... '승용차' '승용차' '승용차']
당사자종별_2당_대분류
['보행자' '승용차' '보행자' ... '보행자' '승용차' '보행자']


In [361]:
print(x_train_cat.shape)
print(x_train_cat['주야'][0].reshape(1))
print(x_train_cat['주야'][0].shape)
x_train_cat

(25037, 16)
[0]
()


,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,사고유형_대분류,사고유형_중분류,법규위반,도로형태_대분류,도로형태,당사자종별_1당_대분류,당사자종별_2당_대분류
0,0,0,1,1,0,0,0,1,93,1,17,10,5,8,5,4
1,0,0,1,4,2,1,0,12,19,2,14,10,5,8,5,7
2,0,0,1,1,0,0,0,15,90,1,13,10,5,8,5,4
3,0,0,1,1,0,0,0,5,87,2,16,10,2,4,5,12
4,1,0,1,2,1,0,0,8,128,2,16,14,5,2,8,15
5,1,0,1,1,0,0,0,4,87,2,16,9,5,8,8,7
6,1,0,1,3,2,0,0,5,54,2,16,1,2,3,5,15
7,1,0,1,1,0,0,0,1,142,1,17,8,5,8,11,4
8,1,0,1,1,0,0,0,15,90,2,16,10,5,8,11,12
9,0,0,1,2,0,1,0,12,25,2,2,10,5,8,11,7


In [362]:
x_train_num

,사망자수,사상자수,중상자수,경상자수,부상신고자수
0,1,1,0,0,0
1,1,4,2,1,0
2,1,1,0,0,0
3,1,1,0,0,0
4,1,2,1,0,0
5,1,1,0,0,0
6,1,3,2,0,0
7,1,1,0,0,0
8,1,1,0,0,0
9,1,2,0,1,0


In [254]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import np_utils

In [258]:
x_train_cat1 = np_utils.to_categorical(x_train_cat)
print(x_train_cat1.shape)
print(x_train_cat1[7][0])

(25037, 16, 209)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
model = Sequential()

model.add()